In [410]:
import pickle
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import shuffle

from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms

train_path = "/Users/dpap/Documents/UvA/AML/Assignments/Kaggle/pickle/"
test_path = "/Users/dpap/Documents/UvA/AML/Assignments/Kaggle/pickle/"

In [113]:
!ls pickle

Archive.zip                test_img.pkl
processed_test_images.pkl  train_fname.pkl
processed_train_images.pkl train_img.pkl
test_fname.pkl             train_lbs.pkl


# Dataset and Dataloaders

In [281]:
train_img = pickle.load(open(train_path+"processed_train_images.pkl","rb"))
labels = pickle.load(open(train_path+"train_lbs.pkl","rb"))

test_img = pickle.load(open(train_path+"processed_test_images.pkl","rb"))
test_fname = pickle.load(open(train_path+"test_fname.pkl","rb"))

In [394]:
test_fname = pickle.load(open(train_path+"test_fname.pkl","rb"))
test_img = pickle.load(open(train_path+"processed_test_images.pkl","rb"))

In [325]:
print(len(train_img))
print(len(tr_labels))
print(len(set(tr_labels)))
print(set(labels))
print(len(test_img))

24204
19363
121
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120}
6132


In [283]:
def shuffle_list(*ls):
    l =list(zip(*ls))
    shuffle(l)
    return zip(*l)
train_img, labels = shuffle_list(train_img, labels)

In [285]:
#seperate dataset to training and test 70-30
sep = int(len(train_img)*0.8)
#training set
tr_images = train_img[:sep]
tr_labels = labels[:sep]
#test set
te_images = train_img[sep:]
te_labels = labels[sep:]

In [287]:
print(len(tr_images))
print(len(tr_labels))

19363
19363


In [288]:
class TrainingImagesDataset(Dataset):
    
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        single_image = self.images[index]
        single_label = self.labels[index]
        
        if self.transform != None:
            #image to tensor
            tensor = self.transform(single_image)
            return (tensor, single_label)
        #return image without turning it to tensor
        #probably wrong
        return (single_image, single_label)
        

In [289]:
class TestingImagesDataset(Dataset):
    
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        single_image = self.images[index]
        if self.transform:
            #image to tensor
            tensor = self.transform(single_image)
            return tensor
        return single_image
        

In [46]:
train_dataset = TrainingImagesDataset(tr_images, tr_labels, transform=transforms.Compose([transforms.Grayscale(), transforms.ToTensor()]))


In [48]:
test_dataset = TrainingImagesDataset(te_images, te_labels, transform=transforms.Compose([transforms.Grayscale(), transforms.ToTensor()]))

In [50]:
len(test_dataset)
print(len(train_dataset)+len(test_dataset))

24204


In [328]:
def create_dataloaders(train_images, train_lbs, test_images=None, test_lbs = None, batch_size = 32):
    
    train_dataset = TrainingImagesDataset(train_images, train_lbs, transform=transforms.Compose([transforms.Grayscale(), transforms.ToTensor()]))
    
    if test_images and test_lbs:
        print("here")
        test_dataset = TrainingImagesDataset(test_images, test_lbs, transform=transforms.Compose([transforms.Grayscale(), transforms.ToTensor()]))
    elif test_images:
        test_dataset = TestingImagesDataset(test_images, transform=transforms.Compose([transforms.Grayscale(), transforms.ToTensor()]))
    else:
        test_dataset = []
    
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers = 0)
    test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, num_workers = 0)    
    
    return (train_loader, test_loader)
    

In [75]:
create_dataloaders(train_img, labels, test_img, 1)

(<torch.utils.data.dataloader.DataLoader at 0x1144b4160>,
 <torch.utils.data.dataloader.DataLoader at 0x1144b4320>)

# Neural Network

In [329]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(2704, 2000)
        self.fc2 = nn.Linear(2000, 2000)
        self.fc3 = nn.Linear(2000, 121)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()


# Training

In [330]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)

In [385]:
train_loader, test_loader = create_dataloaders(tr_images, tr_labels, te_images, te_labels)


here


In [332]:
len(te_labels)

4841

In [355]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, lbs = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, lbs)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 32 == 31:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 32))
            running_loss = 0.0

print('Finished Training')

[1,    32] loss: 0.151
[1,    64] loss: 0.119
[1,    96] loss: 0.087
[1,   128] loss: 0.116
[1,   160] loss: 0.081
[1,   192] loss: 0.099
[1,   224] loss: 0.110
[1,   256] loss: 0.108
[1,   288] loss: 0.088
[1,   320] loss: 0.096
[1,   352] loss: 0.103
[1,   384] loss: 0.084
[1,   416] loss: 0.079
[1,   448] loss: 0.100
[1,   480] loss: 0.123
[1,   512] loss: 0.130
[1,   544] loss: 0.127
[1,   576] loss: 0.143
[2,    32] loss: 0.062
[2,    64] loss: 0.072
[2,    96] loss: 0.079
[2,   128] loss: 0.082
[2,   160] loss: 0.069
[2,   192] loss: 0.068
[2,   224] loss: 0.088
[2,   256] loss: 0.062
[2,   288] loss: 0.089
[2,   320] loss: 0.098
[2,   352] loss: 0.085
[2,   384] loss: 0.093
[2,   416] loss: 0.177
[2,   448] loss: 0.109
[2,   480] loss: 0.116
[2,   512] loss: 0.141
[2,   544] loss: 0.088
[2,   576] loss: 0.098
[3,    32] loss: 0.118
[3,    64] loss: 0.090
[3,    96] loss: 0.056
[3,   128] loss: 0.081
[3,   160] loss: 0.090
[3,   192] loss: 0.068
[3,   224] loss: 0.075
[3,   256] 

In [386]:
correct = 0
total = 0
jj=0
with torch.no_grad():
    try:
        for data in test_loader:
            try:
                jj+=1
                #print("hi")
                images, lbs = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += lbs.size(0)
                correct += (predicted == lbs).sum().item()
                #print("here")
            except:
                print(jj)
    except:
        print(jj)
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
print(correct)
print(total)

Accuracy of the network on the 10000 test images: 54 %
2649
4841


In [387]:
class_correct = list(0. for i in range(121))
class_total = list(0. for i in range(121))
with torch.no_grad():
    for data in test_loader:
        images, lbs = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == lbs).squeeze()
        for i in range(1):
            label = lbs[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(121):
    try:
        #if class_correct[i]:
        #    print (i)
        #    print(class_correct[i])
        #    print(class_total[i])
        print('Accuracy of %5s : %2d %%' % (
            i, 100 * class_correct[i] / class_total[i]))
    except:
        #print(e)
        print(i)
        print(class_correct[i])
        print(class_total[i])

Accuracy of     0 :  0 %
1
0.0
0.0
2
0.0
0.0
Accuracy of     3 : 100 %
Accuracy of     4 : 100 %
Accuracy of     5 : 50 %
Accuracy of     6 : 100 %
Accuracy of     7 : 50 %
Accuracy of     8 : 100 %
9
0.0
0.0
10
0.0
0.0
11
0.0
0.0
Accuracy of    12 : 75 %
13
0.0
0.0
Accuracy of    14 :  0 %
15
0.0
0.0
16
0.0
0.0
17
0.0
0.0
18
0.0
0.0
19
0.0
0.0
Accuracy of    20 :  0 %
Accuracy of    21 : 50 %
Accuracy of    22 :  0 %
Accuracy of    23 :  0 %
Accuracy of    24 : 75 %
Accuracy of    25 :  0 %
Accuracy of    26 : 50 %
Accuracy of    27 : 80 %
Accuracy of    28 : 100 %
Accuracy of    29 :  0 %
Accuracy of    30 :  0 %
Accuracy of    31 :  0 %
Accuracy of    32 : 100 %
33
0.0
0.0
Accuracy of    34 : 50 %
35
0.0
0.0
Accuracy of    36 : 50 %
37
0.0
0.0
Accuracy of    38 :  0 %
39
0.0
0.0
40
0.0
0.0
41
0.0
0.0
Accuracy of    42 :  0 %
Accuracy of    43 :  0 %
44
0.0
0.0
Accuracy of    45 : 42 %
46
0.0
0.0
47
0.0
0.0
48
0.0
0.0
Accuracy of    49 : 100 %
50
0.0
0.0
51
0.0
0.0
52
0.0
0.0
Accurac

# Prediction

In [396]:
test_dataset = TestingImagesDataset(test_img, transform=transforms.Compose([transforms.Grayscale(), transforms.ToTensor()]))
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False, num_workers = 0)

In [416]:
prediction = []
i=0
for images in test_loader:
    i+=1
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    prediction.extend(predicted.numpy())
print(i)


192


In [417]:
results_df = pd.DataFrame({'image': test_fname, 'class': prediction}, columns=['image', 'class'])
results_df.to_csv('results.csv',sep = ',', index = False)

In [418]:
results_df

,image,class
0,42301.jpg,12
1,12536.jpg,8
2,109638.jpg,119
3,84232.jpg,11
4,92450.jpg,7
5,113106.jpg,116
6,117360.jpg,5
7,52674.jpg,8
8,157750.jpg,105
9,32194.jpg,105
